<a href="https://colab.research.google.com/github/Sarthak9504/SY-Sem1-CourseProjects/blob/EDAI-3/Copy_of_Marathi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import re
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.layers import TimeDistributed

In [5]:
df=pd.read_excel("/content/tweets-train-cleaned.xlsx", index_col='Unnamed: 0')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

df.loc[~df['tweet'].isnull()]

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दु...,-1
1,सर्वोच् न्ल निर्देश डावलून पुण पोलिस प्र आनंद ...,-1
2,उद्ध ठाकर भाजपासोबत युत शिसैनिक अमान िश्ासघ फस...,-1
3,आपल समाज खूप मोठ त्य घटक अंतर्भूत त या दिव्यां...,1
4,बलात्कार बदल बलात्कार घेतल पाहिज म्हणणार सावरक...,-1
...,...,...
12109,त्याचबरोबर काज व्यावसायिकां मागील कालावध व्हॅट...,1
12110,सरकार कर्जमाफ योज लाग करुन सुमार वर्ष काळ उलटू...,-1
12111,लोकश मतदार खऱ्य अर्थ सत्ताधार असत धमकािण अथ प्...,-1
12112,छत्रपत शिवाज महाराज ंच् नाव शिवसेने स्थाप ंन श...,-1


In [4]:
df.shape

(12114, 2)

In [5]:
df.loc[~df['tweet'].isnull()]

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दु...,-1
1,सर्वोच् न्ल निर्देश डावलून पुण पोलिस प्र आनंद ...,-1
2,उद्ध ठाकर भाजपासोबत युत शिसैनिक अमान िश्ासघ फस...,-1
3,आपल समाज खूप मोठ त्य घटक अंतर्भूत त या दिव्यां...,1
4,बलात्कार बदल बलात्कार घेतल पाहिज म्हणणार सावरक...,-1
...,...,...
12109,त्याचबरोबर काज व्यावसायिकां मागील कालावध व्हॅट...,1
12110,सरकार कर्जमाफ योज लाग करुन सुमार वर्ष काळ उलटू...,-1
12111,लोकश मतदार खऱ्य अर्थ सत्ताधार असत धमकािण अथ प्...,-1
12112,छत्रपत शिवाज महाराज ंच् नाव शिवसेने स्थाप ंन श...,-1


In [6]:
df_v=pd.read_excel("/content/tweets-valid-cleaned.xlsx", index_col='Unnamed: 0')

In [23]:
df_v.loc[~df_v['tweet'].isnull()]

,tweet,label
0,कोर काळ पंतप्रधान मा नरेंद्र मोदीज य समाज घटक ...,1
1,पुण राष्ट्रसे दल अभ्सिक िभाग ंच् संयुक्त िद्यम...,1
2,पुण क्रांत ज्योत कार्यक्रम जेष्ठ समाजवाद ने मा...,0
3,आद्यक्रांतिकारक उमाजीराज नाईक य जयंतीनिमित्त व...,1
4,महाराष्ट्र मुख्यमंत्र शपथ घेतल्याबद्दल शिवस पक...,1
...,...,...
1495,पोलिसभरत प्रक्रिय जु निकषांनुसार व्व याबाबत मा...,1
1496,निध उद्ध्वस्त कुटुंबांमधील मुल शिक्षण वापरल जा...,1
1497,िकासा फक्त घोषणाबाज जाहिरातबाज राज्य जनते दिशा...,0
1498,विधानसभेत राष्ट्रीय जनगण ओबीसीं इतर मागासवर्गी...,0


In [8]:
df_v.shape

(1500, 2)

In [7]:
df_t = pd.read_excel("/content/tweets-test-cleaned.xlsx", index_col='Unnamed: 0')

In [10]:
df_t.shape

(2250, 2)

In [11]:
max(df_t['tweet'].str.len())

240

In [12]:
max(df_v['tweet'].str.len())

245

In [13]:
num_negative_1 = (df['label'] == -1).sum()
print("Number of data instances under label -1:", num_negative_1)

Number of data instances under label -1: 4038


In [14]:
vocab_df=pd.concat([df_v['tweet'], df.loc[~df['tweet'].isnull(), 'tweet']], axis = 0)

In [15]:
vocab_df.isnull().sum()

0

In [16]:
set2=set()
for row in vocab_df.iteritems():
  for words in row[1].split():
    set2.add(words)
len(set2)

<ipython-input-16-c0937d06b7d9>:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for row in vocab_df.iteritems():


38045

In [17]:

vocab_size=40000


In [18]:
from collections import Counter
def counter_word(text_col):
  count=Counter()
  for text in text_col.values:
    for word in text.split():
      count[word]+=1
  return count

In [19]:

counter = counter_word(vocab_df)

len(counter)


38045

In [20]:
df = df[df['label'] != 2]
df_v = df_v[df_v['label'] != 2]
df_t = df_t[df_t['label'] != 2]

In [21]:

from tensorflow.keras.preprocessing.text import Tokenizer
tokinizer=Tokenizer(num_words=40000)
tokinizer.fit_on_texts(df.loc[~df['tweet'].isnull(), 'tweet'])

In [22]:

word_index = tokinizer.word_index

In [23]:
word_index

{'य': 1,
 'सरकार': 2,
 'महाराष्ट्र': 3,
 'यां': 4,
 'आ': 5,
 'राज्य': 6,
 '…': 7,
 'करण्य': 8,
 'भाजप': 9,
 'कर': 10,
 'काँग्रेस': 11,
 'करत': 12,
 'शुभेच्छ': 13,
 'त्य': 14,
 'त': 15,
 'राज': 16,
 'घेतल': 17,
 'उपस्थित': 18,
 'दि': 19,
 'मा': 20,
 'देश': 21,
 'पक्ष': 22,
 'अभिवादन': 23,
 'सुर': 24,
 'मुंबई': 25,
 'विनम्र': 26,
 'श्र': 27,
 'मुख्यमंत्र': 28,
 'भेट': 29,
 'मराठ': 30,
 'करणार': 31,
 'या': 32,
 'शेतकर': 33,
 'शेतकऱ्य': 34,
 'वर्ष': 35,
 'जन': 36,
 'ने': 37,
 'हार्दिक': 38,
 'श्रद्धांजल': 39,
 'माज': 40,
 'निर्णय': 41,
 'सर्व': 42,
 'लोक': 43,
 'मदत': 44,
 'मंत्र': 45,
 'ना': 46,
 'अस': 47,
 'असत': 48,
 'माझ': 49,
 'च्य': 50,
 'मागण': 51,
 'केंद्र': 52,
 'पाहिज': 53,
 'मोद': 54,
 'महाराज': 55,
 'प्रश्न': 56,
 'साहेब': 57,
 'राष्ट्रवाद': 58,
 'अश': 59,
 'चर्': 60,
 'काळ': 61,
 'भावपूर्ण': 62,
 'भाग': 63,
 'करण': 64,
 'समाज': 65,
 'अध्यक्ष': 66,
 'घेत': 67,
 'कोर': 68,
 'मात्र': 69,
 'बैठक': 70,
 'मोठ': 71,
 'जिल्ह्य': 72,
 'पुण': 73,
 'माझ्य': 74,
 'कराव': 75,
 'ंन': 76,
 '

In [24]:
train_sequences = tokinizer.texts_to_sequences(df.loc[~df['tweet'].isnull(), 'tweet'])

In [25]:
test_sequences = tokinizer.texts_to_sequences(df_t['tweet'].values)


In [26]:
val_sequences = tokinizer.texts_to_sequences(df_v['tweet'].values)


In [27]:

print(df['tweet'][0])
print(train_sequences[0])

ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दुखद दशकं परखड लेखण पत्रकारितेत अमूल्य योगदान दि दीक्षित यां मार्गदर्शनाखाल पत्रकारां पिढ घडल अनंत दीक्षित य भावपूर्ण श्रद्धांजल त्य परिवार दुख सहभाग आहोत
[83, 132, 1800, 4171, 4, 77, 795, 1676, 8272, 2692, 2387, 10711, 2693, 268, 19, 4171, 4, 1864, 5150, 994, 1732, 1800, 4171, 1, 62, 39, 14, 1733, 1230, 97, 236]


In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
train_padded=pad_sequences(train_sequences, maxlen=250, padding='post', truncating='post')
val_padded=pad_sequences(val_sequences, maxlen=250, padding='post', truncating='post')
test_padded=pad_sequences(test_sequences, maxlen=250, padding='post', truncating='post')

In [30]:
print(train_padded.shape, val_padded.shape, test_padded.shape)

(12112, 250) (1500, 250) (2250, 250)


In [31]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
from keras.optimizers import Adam
model = Sequential()

In [32]:
model.add(layers.Embedding(40000, 32, input_length=250))
model.add(layers.Dense(256, activation='gelu'))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))

In [33]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0002)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])


In [34]:
train_label_categorical = to_categorical(df.loc[~df['tweet'].isnull(), 'label'], num_classes=3)


val_label_categorical = to_categorical(df_v['label'].values, num_classes=3)


test_label_categorical = to_categorical(df_t['label'].values, num_classes=3)

In [35]:

val_label_categorical

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [36]:
train_label_categorical

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [37]:
tf.config.run_functions_eagerly(True)
model.fit(train_padded, to_categorical(df.loc[~df['tweet'].isnull(), 'label'].values, num_classes=3), epochs=5, validation_data=(val_padded, to_categorical(df_v['label'].values, num_classes=3)), verbose=2)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


379/379 - 18s - loss: 1.0106 - accuracy: 0.4668 - val_loss: 0.8072 - val_accuracy: 0.6653 - 18s/epoch - 48ms/step
Epoch 2/5
379/379 - 15s - loss: 0.6064 - accuracy: 0.7655 - val_loss: 0.5465 - val_accuracy: 0.7673 - 15s/epoch - 39ms/step
Epoch 3/5
379/379 - 13s - loss: 0.3693 - accuracy: 0.8594 - val_loss: 0.5335 - val_accuracy: 0.7813 - 13s/epoch - 36ms/step
Epoch 4/5
379/379 - 13s - loss: 0.2372 - accuracy: 0.9172 - val_loss: 0.5476 - val_accuracy: 0.7820 - 13s/epoch - 34ms/step
Epoch 5/5
379/379 - 13s - loss: 0.1481 - accuracy: 0.9558 - val_loss: 0.5991 - val_accuracy: 0.7820 - 13s/epoch - 35ms/step


In [38]:

model.evaluate(test_padded, to_categorical(df_t['label'].values, num_classes=3))

71/71 [==============================] - 1s 14ms/step - loss: 0.6091 - accuracy: 0.7729


[0.6090817451477051, 0.7728888988494873]

In [39]:
predictions = model.predict(test_padded)
predictions


71/71 [==============================] - 0s 5ms/step


array([[5.1333626e-07, 1.0126209e-06, 9.9999845e-01],
       [5.5743754e-03, 1.0671204e-01, 8.8771361e-01],
       [2.3223541e-03, 2.5230086e-01, 7.4537677e-01],
       ...,
       [9.7611815e-01, 1.2811517e-02, 1.1070386e-02],
       [5.3298402e-01, 4.0719318e-01, 5.9822790e-02],
       [1.7324080e-01, 8.1247568e-01, 1.4283495e-02]], dtype=float32)

In [40]:
import sklearn
matrix = sklearn.metrics.confusion_matrix(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1))
print(matrix)

[[554 109  87]
 [114 580  56]
 [ 95  50 605]]


In [41]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1)))
print(f1_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1), average=None))


0.7728888888888888
[0.73231989 0.77904634 0.80774366]


In [42]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00


In [43]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=54c2e35cba4bf56629f7aeaa57d587267d9dd8f038f5ac7dcc36d38a506aa735
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [44]:
from transformers import AutoTokenizer, BertTokenizer, TFBertModel, AutoModel, TFAutoModel, TFAutoModelForSequenceClassification

In [45]:
from sentence_transformers import SentenceTransformer

In [46]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/indic-sentence-bert-nli")

In [47]:
df = pd.read_excel("/content/tweets-train-cleaned.xlsx", index_col='Unnamed: 0')
df_v = pd.read_excel("/content/tweets-valid-cleaned.xlsx", index_col='Unnamed: 0')
df_t = pd.read_excel("/content/tweets-test-cleaned.xlsx", index_col='Unnamed: 0')


In [48]:
train_inputs = tokenizer(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")
val_inputs = tokenizer(df_v['tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")
test_inputs = tokenizer(df_t['tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")

In [19]:
train_labels = df.loc[~df['tweet'].isnull(), 'label'].values
val_labels = df_v['label'].values
test_labels = df_t['label'].values

In [50]:
num_classes = 3
train_label_categorical = to_categorical(train_labels, num_classes=num_classes)
val_label_categorical = to_categorical(val_labels, num_classes=num_classes)
test_label_categorical = to_categorical(test_labels, num_classes=num_classes)

In [51]:
model = SentenceTransformer("l3cube-pune/indic-sentence-bert-nli")

In [52]:
train_embeddings = model.encode(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), show_progress_bar=True)
val_embeddings = model.encode(df_v['tweet'].tolist(), show_progress_bar=True)
test_embeddings = model.encode(df_t['tweet'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/379 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [53]:
classification_model = tf.keras.Sequential([
    Dense(256, activation='relu', input_shape=(train_embeddings.shape[1],)),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

In [87]:
classification_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 10, 256)          918528    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 10, 256)           65792     
                                                                 
 dropout_2 (Dropout)         (None, 10, 256)           0         
                                                                 
 dense_7 (Dense)             (None, 10, 3)             771       
                                                                 
Total params: 985,091
Trainable params: 985,091
Non-trainable params: 0
_________________________________________________________________


In [54]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0002)
classification_model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])


In [55]:
classification_model.fit(
    train_embeddings,
    train_label_categorical,
    epochs=9,
    batch_size=32,
    validation_data=(val_embeddings, val_label_categorical),
    verbose=2
)

Epoch 1/9


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


379/379 - 11s - loss: 0.8678 - accuracy: 0.6844 - val_loss: 0.6718 - val_accuracy: 0.7433 - 11s/epoch - 30ms/step
Epoch 2/9
379/379 - 11s - loss: 0.6531 - accuracy: 0.7313 - val_loss: 0.5969 - val_accuracy: 0.7613 - 11s/epoch - 29ms/step
Epoch 3/9
379/379 - 10s - loss: 0.6167 - accuracy: 0.7420 - val_loss: 0.5784 - val_accuracy: 0.7620 - 10s/epoch - 26ms/step
Epoch 4/9
379/379 - 12s - loss: 0.6024 - accuracy: 0.7472 - val_loss: 0.5693 - val_accuracy: 0.7667 - 12s/epoch - 31ms/step
Epoch 5/9
379/379 - 25s - loss: 0.5920 - accuracy: 0.7535 - val_loss: 0.5616 - val_accuracy: 0.7753 - 25s/epoch - 65ms/step
Epoch 6/9
379/379 - 21s - loss: 0.5861 - accuracy: 0.7564 - val_loss: 0.5571 - val_accuracy: 0.7727 - 21s/epoch - 54ms/step
Epoch 7/9
379/379 - 24s - loss: 0.5780 - accuracy: 0.7617 - val_loss: 0.5554 - val_accuracy: 0.7733 - 24s/epoch - 63ms/step
Epoch 8/9
379/379 - 17s - loss: 0.5708 - accuracy: 0.7612 - val_loss: 0.5473 - val_accuracy: 0.7760 - 17s/epoch - 44ms/step
Epoch 9/9
379/379 

In [56]:
test_loss, test_acc = classification_model.evaluate(test_embeddings, test_label_categorical)
print("Test accuracy:", test_acc)

71/71 [==============================] - 1s 16ms/step - loss: 0.5666 - accuracy: 0.7676
Test accuracy: 0.7675555348396301


In [57]:
predictions = classification_model.predict(test_embeddings)
predicted_labels = np.argmax(predictions, axis=1)
predictions

71/71 [==============================] - 0s 4ms/step


array([[0.03029607, 0.04420022, 0.9255037 ],
       [0.1671262 , 0.05798945, 0.77488434],
       [0.2007192 , 0.5817262 , 0.21755457],
       ...,
       [0.539519  , 0.08757219, 0.3729087 ],
       [0.78925335, 0.10767736, 0.10306924],
       [0.10777646, 0.858783  , 0.03344058]], dtype=float32)

In [58]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       750
           0       0.73      0.73      0.73       750
           1       0.78      0.76      0.77       750
           2       0.00      0.00      0.00         0

    accuracy                           0.50      2250
   macro avg       0.38      0.37      0.37      2250
weighted avg       0.50      0.50      0.50      2250



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [59]:
predicted_labels = np.argmax(predictions, axis=1)

In [60]:
conf_matrix = confusion_matrix(test_labels, predicted_labels, labels=[0, 1, 2])

In [66]:
import sklearn
matrix = sklearn.metrics.confusion_matrix(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1))
print(matrix)

[[551 105  94]
 [122 568  60]
 [ 86  56 608]]


In [67]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1)))
print(f1_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1),average=None))

0.7675555555555555
[0.73028496 0.76808654 0.8042328 ]


In [64]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [14]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mr.300.bin.gz


--2023-08-18 21:03:51--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mr.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.70, 13.227.219.33, 13.227.219.10, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3220048823 (3.0G) [application/octet-stream]
Saving to: ‘cc.mr.300.bin.gz’

cc.mr.300.bin.gz    100%[===================>]   3.00G  20.9MB/s    in 1m 48s  

2023-08-18 21:05:40 (28.3 MB/s) - ‘cc.mr.300.bin.gz’ saved [3220048823/3220048823]



In [15]:
!gunzip cc.mr.300.bin.gz

In [39]:
!pip install fasttext


In [1]:
import fasttext

In [2]:
fasttext = fasttext.load_model('cc.mr.300.bin')

In [8]:
train_text = df.loc[~df['tweet'].isnull(), 'tweet']
val_text = df_v['tweet']
test_text = df_t['tweet']

In [9]:
tokenizer = Tokenizer(num_words=40000)
tokenizer.fit_on_texts(train_text)

In [10]:
train_sequences = tokenizer.texts_to_sequences(train_text)
val_sequences = tokenizer.texts_to_sequences(val_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

In [11]:
max_sequence_length = 250
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
val_padded = pad_sequences(val_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [28]:
embedding_dim = fasttext.get_dimension()
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in fasttext.words:
        embedding_matrix[i] = fasttext[word]

In [60]:
print(embedding_matrix.shape)

(35666, 300)


In [67]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

In [68]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [69]:
train_label_categorical = to_categorical(train_labels, num_classes=3)
val_label_categorical = to_categorical(val_labels, num_classes=3)
test_label_categorical = to_categorical(test_labels, num_classes=3)

In [70]:
print("embedding_matrix shape:", embedding_matrix.shape)
print("train_padded shape:", train_padded.shape)
print("val_padded shape:", val_padded.shape)
print("test_padded shape:", test_padded.shape)
print("train_label_categorical shape:", train_label_categorical.shape)
print("val_label_categorical shape:", val_label_categorical.shape)
print("test_label_categorical shape:", test_label_categorical.shape)

embedding_matrix shape: (35666, 300)
train_padded shape: (12112, 250)
val_padded shape: (1500, 250)
test_padded shape: (2250, 250)
train_label_categorical shape: (12112, 3)
val_label_categorical shape: (1500, 3)
test_label_categorical shape: (2250, 3)


In [71]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 250, 300)          10699800  
                                                                 
 bidirectional_7 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dense_15 (Dense)            (None, 256)               65792     
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_16 (Dense)            (None, 3)                 771       
                                                                 
Total params: 11,205,659
Trainable params: 505,859
Non-trainable params: 10,699,800
____________________________________

In [74]:
model.fit(train_padded, train_label_categorical,
          epochs=8,
          validation_data=(val_padded, val_label_categorical),
          verbose=2)


Epoch 1/8
379/379 - 9s - loss: 0.5101 - accuracy: 0.7948 - val_loss: 0.6382 - val_accuracy: 0.7400 - 9s/epoch - 24ms/step
Epoch 2/8
379/379 - 8s - loss: 0.5114 - accuracy: 0.7890 - val_loss: 0.6289 - val_accuracy: 0.7447 - 8s/epoch - 22ms/step
Epoch 3/8
379/379 - 8s - loss: 0.4980 - accuracy: 0.8014 - val_loss: 0.6261 - val_accuracy: 0.7440 - 8s/epoch - 20ms/step
Epoch 4/8
379/379 - 8s - loss: 0.4856 - accuracy: 0.8042 - val_loss: 0.6482 - val_accuracy: 0.7327 - 8s/epoch - 22ms/step
Epoch 5/8
379/379 - 8s - loss: 0.4821 - accuracy: 0.8068 - val_loss: 0.6462 - val_accuracy: 0.7413 - 8s/epoch - 20ms/step
Epoch 6/8
379/379 - 8s - loss: 0.4677 - accuracy: 0.8123 - val_loss: 0.6577 - val_accuracy: 0.7433 - 8s/epoch - 22ms/step
Epoch 7/8
379/379 - 8s - loss: 0.4591 - accuracy: 0.8158 - val_loss: 0.6383 - val_accuracy: 0.7420 - 8s/epoch - 20ms/step
Epoch 8/8
379/379 - 8s - loss: 0.4517 - accuracy: 0.8199 - val_loss: 0.6645 - val_accuracy: 0.7340 - 8s/epoch - 21ms/step


In [75]:
test_loss, test_acc = model.evaluate(test_padded, test_label_categorical)
print("Test accuracy:", test_acc)

71/71 [==============================] - 1s 12ms/step - loss: 0.6410 - accuracy: 0.7564
Test accuracy: 0.7564444541931152


In [76]:
predictions = model.predict(test_padded)
predicted_labels = np.argmax(predictions, axis=1)

71/71 [==============================] - 2s 12ms/step


In [77]:
print(classification_report(test_labels, predicted_labels))
conf_matrix = confusion_matrix(test_labels, predicted_labels, labels=[0, 1, 2])

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       750
           0       0.71      0.73      0.72       750
           1       0.78      0.72      0.75       750
           2       0.00      0.00      0.00         0

    accuracy                           0.48      2250
   macro avg       0.37      0.36      0.37      2250
weighted avg       0.50      0.48      0.49      2250

Confusion Matrix:
[[549 102  99]
 [132 541  77]
 [  0   0   0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [78]:
import sklearn
matrix = sklearn.metrics.confusion_matrix(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1))
print(matrix)


[[549 102  99]
 [132 541  77]
 [ 87  51 612]]


In [80]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1)))
print(f1_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1),average=None))

0.7564444444444445
[0.72332016 0.74930748 0.79583875]
